In [30]:
import pandas as pd
import numpy as np

df = pd.read_csv("application_train.csv")



In [31]:
#convert Employment tenure
EMPLOYMENT_YEARS = df['DAYS_EMPLOYED'] / -365.25
EMPLOYMENT_YEARS

0           1.744011
1           3.252567
2           0.616016
3           8.320329
4           8.317591
             ...    
307506      0.646133
307507   -999.980835
307508     21.686516
307509     13.103354
307510      3.455168
Name: DAYS_EMPLOYED, Length: 307511, dtype: float64

In [32]:
#Create ratios:
DTI = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
LOAN_TO_INCOME = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
ANNUITY_TO_CREDIT = df['AMT_ANNUITY'] / df['AMT_CREDIT']

In [33]:
# Handle missing values (report % and apply strategy: drop columns > 60% missing; impute median/most-frequent for others)
missing_values_pct=(df.isnull().mean())*100
missing_values_pct
df=df.loc[:,missing_values_pct>=60]
df

,OWN_CAR_AGE,YEARS_BUILD_AVG,COMMONAREA_AVG,FLOORSMIN_AVG,LIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_AVG,YEARS_BUILD_MODE,COMMONAREA_MODE,FLOORSMIN_MODE,LIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_MODE,YEARS_BUILD_MEDI,COMMONAREA_MEDI,FLOORSMIN_MEDI,LIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MEDI,FONDKAPREMONT_MODE
0,NaN,0.6192,0.0143,0.1250,0.0202,0.0000,0.6341,0.0144,0.1250,0.0220,0.0,0.6243,0.0144,0.1250,0.0205,0.0000,reg oper account
1,NaN,0.7960,0.0605,0.3333,0.0773,0.0039,0.8040,0.0497,0.3333,0.0790,0.0,0.7987,0.0608,0.3333,0.0787,0.0039,reg oper account
2,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,NaN,0.8300,0.0202,0.2708,0.1484,0.0753,0.7125,0.0172,0.0417,0.0882,0.0,0.8323,0.0203,0.2708,0.1509,0.0757,reg oper account
307507,NaN,0.6260,0.0022,0.1250,0.0202,0.0000,0.6406,0.0022,0.1250,0.0220,0.0,0.6310,0.0022,0.1250,0.0205,0.0000,reg oper account
307508,NaN,0.7484,0.0123,0.2083,0.0841,0.0000,0.7583,0.0124,0.2083,0.0918,0.0,0.7518,0.0124,0.2083,0.0855,0.0000,reg oper account
307509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
EMPLOYMENT_YEARS=df['DAYS_EMPLOYED']{lambda x: -x/365.25 if x<=0 print null }



SyntaxError: expected 'else' after 'if' expression (1965321185.py, line 1)

In [ ]:
missing_values = df.isnull().mean() * 100
missing_values
print(missing_values[missing_values>=60].index.tolist())
drop_cols = missing_values[missing_values>=60].index.tolist()
df = df.drop(columns=drop_cols)
df.shape

['OWN_CAR_AGE', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'FLOORSMIN_AVG', 'LIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_AVG', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'FLOORSMIN_MODE', 'LIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_MODE', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'FLOORSMIN_MEDI', 'LIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'FONDKAPREMONT_MODE']


(307511, 0)

In [ ]:
import numpy as np
import pandas as pd

def optimize_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Optimize numeric dtypes:
      - Integers → int8 / int16 / int32
      - Floats   → float16 / float32
    Returns a DataFrame with reduced memory usage.
    """
    before = df.memory_usage(deep=True).sum() / 1024**2
    optimized = df.copy()

    for col in optimized.columns:
        s = optimized[col]

        # --- Integers ---
        if pd.api.types.is_integer_dtype(s):
            vals = s.astype("int64")  # safe for bound checks
            if vals.min() >= np.iinfo(np.int8).min and vals.max() <= np.iinfo(np.int8).max:
                optimized[col] = s.astype("int8")
            elif vals.min() >= np.iinfo(np.int16).min and vals.max() <= np.iinfo(np.int16).max:
                optimized[col] = s.astype("int16")
            elif vals.min() >= np.iinfo(np.int32).min and vals.max() <= np.iinfo(np.int32).max:
                optimized[col] = s.astype("int32")
            # else: leave unchanged if too large

        # --- Floats ---
        elif pd.api.types.is_float_dtype(s):
            s64 = s.astype("float64")
            if np.allclose(s64, s64.astype("float16"), rtol=1e-03, atol=1e-06, equal_nan=True):
                optimized[col] = s64.astype("float16")
            else:
                optimized[col] = s64.astype("float32")

    after = optimized.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory: {before:.3f} MB → {after:.3f} MB ({(before - after) / before * 100:.2f}% reduction)")

    return optimized.shape 


In [ ]:
import numpy as np
import pandas as pd

def optimize_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Optimize numeric dtypes:
      - Integers → int8 / int16 / int32
      - Floats   → float16 / float32
    Returns a DataFrame with reduced memory usage.
    """
    before = df.memory_usage(deep=True).sum() / 1024**2
    optimized = df.copy()

    for col in optimized.columns:
        s = optimized[col]

        # --- Integers ---
        if pd.api.types.is_integer_dtype(s):
            vals = s.astype("int64")  # safe for bound checks
            if vals.min() >= np.iinfo(np.int8).min and vals.max() <= np.iinfo(np.int8).max:
                optimized[col] = s.astype("int8")
            elif vals.min() >= np.iinfo(np.int16).min and vals.max() <= np.iinfo(np.int16).max:
                optimized[col] = s.astype("int16")
            elif vals.min() >= np.iinfo(np.int32).min and vals.max() <= np.iinfo(np.int32).max:
                optimized[col] = s.astype("int32")

        # --- Floats ---
        elif pd.api.types.is_float_dtype(s):
            s64 = s.astype("float64")
            if np.allclose(s64, s64.astype("float16"), rtol=1e-03, atol=1e-06, equal_nan=True):
                optimized[col] = s64.astype("float16")
            else:
                optimized[col] = s64.astype("float32")

    after = optimized.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory: {before:.3f} MB → {after:.3f} MB ({(before - after) / before * 100:.2f}% reduction)")

    return optimized  # ✅ return DataFrame instead of memory usage


def treat_nulls(df: pd.DataFrame) -> pd.DataFrame:
    """
    Treat null values in the DataFrame:
      - Drop columns with >60% nulls
      - Fill numeric columns with median
      - Fill categorical/object columns with mode
      - Fill datetime columns with forward fill, then backward fill
    Returns a cleaned DataFrame.
    """
    cleaned = optimize_dataframe(df)

    # 1. Drop columns with >60% nulls
    threshold = 0.6
    null_ratio = cleaned.isna().mean()
    to_drop = null_ratio[null_ratio > threshold].index
    cleaned = cleaned.drop(columns=to_drop)

    # 2. Handle remaining nulls
    for col in cleaned.columns:
        if cleaned[col].isna().any():
            if pd.api.types.is_numeric_dtype(cleaned[col]):
                median_val = cleaned[col].median()
                cleaned[col] = cleaned[col].fillna(median_val)

            elif pd.api.types.is_datetime64_any_dtype(cleaned[col]):
                cleaned[col] = cleaned[col].fillna(method="ffill").fillna(method="bfill")

            else:
                mode_val = cleaned[col].mode(dropna=True)
                if not mode_val.empty:
                    cleaned[col] = cleaned[col].fillna(mode_val[0])
                else:
                    cleaned[col] = cleaned[col].fillna("")

    return cleaned
# df = pd.read_csv("application_train.csv")   # load raw dataset
cleaned_df = treat_nulls(df)  


c:\Users\haris\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\haris\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\haris\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\haris\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


Memory: 504.985 MB → 311.137 MB (38.39% reduction)


In [38]:
cleaned_df['Employe worked'] = cleaned_df['DAYS_EMPLOYED'].apply(lambda x: x / 365.25 if x <= 0 else np.nan)

0         1
1         0
2         0
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307511, dtype: int8